<a href="https://colab.research.google.com/github/Svelez97/ALGORITMO-de-pre-visualizacion-de-datos/blob/master/Problemas_con_la_regresi%C3%B3n_lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%config IPCompleter.greedy=True
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

from google.colab import drive
drive.mount('/content/drive')
MainPath = "/content/drive/MyDrive/Curso MAchine Learning/python-ml-course/datasets/" # Raiz del dataset.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
archivo = 'ecom-expense/Ecom Expense.csv' # Puntero del archivo específico con extensión
ruta = MainPath + archivo
df = pd.read_csv(ruta)
tamano = len(df)
## previsualizacion del dataset para verificar que si se abrio el archivo correcto
print(tamano)
df.head()


2362


,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485


In [ ]:
dummy_gender = pd.get_dummies(df["Gender"], prefix = "Gender")
dummy_city_tier = pd.get_dummies(df["City Tier"], prefix = "City")

In [ ]:
dummy_gender.head()

,Gender_Female,Gender_Male
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [ ]:
dummy_city_tier.head()

,City_Tier 1,City_Tier 2,City_Tier 3
0,1,0,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


## **Ahora se busca anexar las nuevas columnas a todo el dataset**

In [ ]:
column_names = df.columns.values.tolist()
column_names

['Transaction ID',
 'Age ',
 ' Items ',
 'Monthly Income',
 'Transaction Time',
 'Record',
 'Gender',
 'City Tier',
 'Total Spend']

In [ ]:
df_new = df[column_names].join(dummy_gender)
column_names = df_new.columns.values.tolist()
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0


In [ ]:
df_new = df_new[column_names].join(dummy_city_tier)
column_names = df_new.columns.values.tolist()
print(column_names)
print(len(df_new))
df_new.head()

['Transaction ID', 'Age ', ' Items ', 'Monthly Income', 'Transaction Time', 'Record', 'Gender', 'City Tier', 'Total Spend', 'Gender_Female', 'Gender_Male', 'City_Tier 1', 'City_Tier 2', 'City_Tier 3']
2362


,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0


## **Ahora se hace un procedimiento con la libreria scikit-learn para ver cuales son los parametros óptimos para hacer la regresion lineal**

In [ ]:
feature_cols = []
feature_ot_number = ["Total Spend","Transaction ID","Gender","City Tier"]
for feature in column_names:
  if feature in feature_ot_number:
    continue
  else:
    feature_cols.append(feature)

print(feature_cols)

X = df_new[feature_cols]
Y = df_new["Total Spend"]

estimator = SVR(kernel="linear")
selector = RFE(estimator, n_features_to_select=8, step=1)
selector = selector.fit(X,Y)

['Age ', ' Items ', 'Monthly Income', 'Transaction Time', 'Record', 'Gender_Female', 'Gender_Male', 'City_Tier 1', 'City_Tier 2', 'City_Tier 3']


In [ ]:
selector.support_[0]
dicc = {}
k = 0
feature_cols_2 = []

for feature in feature_cols:
  dicc[feature] = selector.support_[k]
  print(feature, selector.support_[k])
  k += 1


for feature_2 in feature_cols:
  if dicc[feature_2]:
    feature_cols_2.append(feature_2)


print(feature_cols_2)


Age  True
 Items  True
Record False
Gender_Female False
Gender_Male True
City_Tier 1 True
City_Tier 2 True
City_Tier 3 True
['Age ', ' Items ', 'Gender_Male', 'City_Tier 1', 'City_Tier 2', 'City_Tier 3']


In [ ]:
'''
feature_cols = ["Monthly Income", "Transaction Time", 
                "Gender_Female", "Gender_Male", 
                "City_Tier 1", "City_Tier 2", "City_Tier 3",
                "Record"]
'''                

'\nfeature_cols = ["Monthly Income", "Transaction Time", \n                "Gender_Female", "Gender_Male", \n                "City_Tier 1", "City_Tier 2", "City_Tier 3",\n                "Record"]\n'

In [ ]:
feature_cols = feature_cols_2
X = df_new[feature_cols]
Y = df_new["Total Spend"]

In [ ]:
lm = LinearRegression()
lm.fit(X,Y)

LinearRegression()

In [ ]:
print(lm.intercept_)
print(lm.coef_[0])

1752.6703433599641
7.217973758190679


In [ ]:
list(zip(feature_cols, lm.coef_))

[('Age ', 7.217973758190679),
 (' Items ', 35.53429689906477),
 ('Record', 778.5703890572173),
 ('Gender_Female', -140.91050076048677),
 ('Gender_Male', 140.91050076048663),
 ('City_Tier 1', 44.572920057345314),
 ('City_Tier 2', 42.77224771780006),
 ('City_Tier 3', -87.34516777514531)]

In [ ]:
lm.score(X,Y)

0.7426512282586561

In [ ]:
valor_inicial = lm.intercept_
i = 0

while i < len(df_new):
  j = 0
  resultado = 0
  for feature in feature_cols:
    resultado += df_new[feature]*lm.coef_[j]
    j += 1

  df_new["prediction"] = valor_inicial + resultado
  i += 1

# df_new["prediction"] = -79.41713030137362 + df_new['Monthly Income']*0.14753898049205738 + df_new['Transaction Time']* 0.15494612549589545+ df_new['Gender_Female'] * (-131.02501325554567) + df_new['Gender_Male'] * 131.0250132555456+ df_new['City_Tier 1']* 76.76432601049527 +  df_new['City_Tier 2']* 55.138974309232474 + df_new['City_Tier 3']* (-131.9033003197278)+ df_new['Record']* 772.2334457445648

In [ ]:
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3,prediction
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0,6207.682575
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0,4447.749003
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0,4223.615902
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0,7858.101440
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0,3715.819888


In [ ]:
SSD = np.sum((df_new["prediction"] - df_new["Total Spend"])**2)
RSE = np.sqrt(SSD/(len(df_new)-len(feature_cols)-1))
sales_mean=np.mean(df_new["Total Spend"])
error = 100*(RSE/sales_mean)
print(SSD)
print(RSE)
print(sales_mean)
print(error)


4762811981.322928
1422.7243192024355
6163.176415976715
23.08427056402811
